# PROCESAMIENTO NATURAL DEL LENGUAJE

Este cuaderno cubre los capítulos 22 y 23 del libro *Inteligencia artificial: un enfoque moderno*, tercera edición. Las implementaciones de los algoritmos se pueden encontrar en [nlp.py](https://github.com/aimacode/aima-python/blob/master/nlp.py).

Ejecute la siguiente celda para importar el código del módulo y comenzar.

In [1]:
import nlp
from nlp import Page, HITS
from nlp import Lexicon, Rules, Grammar, ProbLexicon, ProbRules, ProbGrammar
from nlp import CYK_parse, Chart

from notebook import psource

## CONTENIDO

* Descripción general
* Idiomas
* GOLPES
* Respuesta a preguntas
* Análisis CYK
* Análisis de gráficos

## DESCRIPCIÓN GENERAL

**El procesamiento del lenguaje natural (PNL)** es un campo de la IA que se ocupa de comprender, analizar y utilizar lenguajes naturales. Este campo se considera un campo de estudio difícil pero intrigante, ya que está relacionado con el funcionamiento de los humanos y sus lenguajes.

Las aplicaciones del campo incluyen traducción, reconocimiento de voz, segmentación de temas, extracción y recuperación de información y mucho más.

A continuación echamos un vistazo a algunos algoritmos en el campo. Sin embargo, antes de entrar en materia, echaremos un vistazo a una forma de lenguaje muy útil: los lenguajes **libres de contexto**. Aunque son un poco restrictivos, se han utilizado mucho en investigaciones sobre el procesamiento del lenguaje natural.

## IDIOMAS

Los idiomas pueden representarse mediante un conjunto de reglas gramaticales sobre un léxico de palabras. Diferentes idiomas pueden representarse mediante diferentes tipos de gramática, pero en Procesamiento del Lenguaje Natural nos interesan principalmente las gramáticas libres de contexto.

### Gramáticas libres de contexto

Muchos lenguajes naturales y de programación se pueden representar mediante una **Gramática libre de contexto (CFG)**. Un CFG es una gramática que tiene un único símbolo no terminal en el lado izquierdo. Eso significa que un no terminal puede reemplazarse por el lado derecho de la regla independientemente del contexto. Un ejemplo de CFG:

```
S -> aSb | ε
```

Eso significa que `S` se puede reemplazar por `aSb` o `ε` (con `ε` denotamos la cadena vacía). El léxico del idioma está compuesto por las terminales `a` y `b`, mientras que con `S` denotamos el símbolo no terminal. En general, los no terminales están en mayúscula, mientras que los terminales no, y normalmente denominamos al no terminal inicial "S". El lenguaje generado por la gramática anterior es el lenguaje a<sup>n</sup>b<sup>n</sup> para n mayor o igual que 1.

### Gramática probabilística libre de contexto

Si bien un CFG simple puede resultar muy útil, es posible que deseemos conocer la probabilidad de que se produzca cada regla. Arriba, no sabemos si es más probable que "S" sea reemplazado por "aSb" o "ε". **Las gramáticas probabilísticas libres de contexto (PCFG)** están diseñadas para satisfacer exactamente esa necesidad. Cada regla tiene una probabilidad, dada entre paréntesis, y las probabilidades de una regla suman 1:

```
S -> aSb [0.7] | ε [0.3]
```

Ahora sabemos que es más probable que "S" sea reemplazado por "aSb" que por "ε".

Un problema con los *PCFG* es cómo asignaremos las distintas probabilidades a las reglas. Podríamos utilizar nuestro conocimiento como humanos para asignar las probabilidades, pero esa es una tarea laboriosa y propensa a errores. En cambio, podemos *aprender* las probabilidades a partir de los datos. Los datos se clasifican como etiquetados (con oraciones correctamente analizadas, generalmente denominadas **banco de árboles**) o sin etiquetar (con solo nombres de categorías léxicas y sintácticas).

Con datos etiquetados, podemos simplemente contar las ocurrencias. Para la gramática anterior, si tenemos 100 reglas `S` y 30 de ellas son de la forma `S -> ε`, asignamos una probabilidad de 0,3 a la transformación.

Con datos sin etiquetar tenemos que aprender tanto las reglas gramaticales como la probabilidad de cada regla. Podemos utilizar muchos enfoques, uno de ellos, el algoritmo **adentro-afuera**. Utiliza un enfoque de programación dinámica, que primero encuentra la probabilidad de que cada regla genere una subcadena y luego estima la probabilidad de cada regla.

### Forma normal de Chomsky

Una gramática está en forma normal de Chomsky (o **CNF**, que no debe confundirse con *forma normal conjuntiva*) si sus reglas son una de las tres:

* `X -> Y Z`
*`A->a`
*`S->e`

Donde *X*, *Y*, *Z*, *A* son no terminales, *a* es un terminal, *ε* es la cadena vacía y *S* es el símbolo inicial (el símbolo inicial no debe ser que aparece en el lado derecho de las reglas). Tenga en cuenta que puede haber varias reglas para cada no terminal del lado izquierdo, siempre que sigan lo anterior. Por ejemplo, una regla para *X* podría ser: `X -> Y Z | A B | un | b`.

Por supuesto, también podemos tener un *CNF* con probabilidades.

Este tipo de gramática puede parecer restrictiva, pero se puede demostrar que cualquier gramática libre de contexto se puede convertir a CNF.

### Léxico

El léxico de un idioma se define como una lista de palabras permitidas. Estas palabras se agrupan en las clases habituales: `verbos`, `sustantivos`, `adjetivos`, `adverbios`, `pronombres`, `nombres`, `artículos`, `preposiciones` y `conjunciones`. Para las primeras cinco clases es imposible enumerar todas las palabras, ya que continuamente se agregan palabras en las clases. Recientemente se agregó "google" a la lista de verbos, y palabras como esa seguirán apareciendo y agregándose a las listas. Por esa razón, estas primeras cinco categorías se denominan **clases abiertas**. El resto de categorías tienen muchas menos palabras y mucho menos desarrollo. Si bien palabras como "tú" se usaban comúnmente en el pasado, pero su uso ha disminuido casi por completo, la mayoría de los cambios tardan muchas décadas o siglos en manifestarse, por lo que podemos asumir con seguridad que las categorías permanecerán estáticas en el futuro previsible. Por lo tanto, estas categorías se denominan **clases cerradas**.

Un léxico de ejemplo para un PCFG (tenga en cuenta que también se pueden usar otras clases según el idioma, como `dígitos` o `RelPro` para pronombre relativo):

```
Verb -> is [0.3] | say [0.1] | are [0.1] | ...
Noun -> robot [0.1] | sheep [0.05] | fence [0.05] | ...
Adjective -> good [0.1] | new [0.1] | sad [0.05] | ...
Adverb -> here [0.1] | lightly [0.05] | now [0.05] | ...
Pronoun -> me [0.1] | you [0.1] | he [0.05] | ...
RelPro -> that [0.4] | who [0.2] | which [0.2] | ...
Name -> john [0.05] | mary [0.05] | peter [0.01] | ...
Article -> the [0.35] | a [0.25] | an [0.025] | ...
Preposition -> to [0.25] | in [0.2] | at [0.1] | ...
Conjunction -> and [0.5] | or [0.2] | but [0.2] | ...
Digit -> 1 [0.3] | 2 [0.2] | 0 [0.2] | ...
```

### Gramática

Con las gramáticas combinamos palabras del léxico en frases válidas. Una gramática se compone de **reglas gramaticales**. Cada regla transforma el lado izquierdo de la regla en el lado derecho. Por ejemplo, `A -> B` significa que `A` se transforma en `B`. Construyamos una gramática para el idioma que comenzamos a construir con el léxico. Usaremos un PCFG.

```
S -> NP VP [0.9] | S Conjunction S [0.1]

NP -> Pronoun [0.3] | Name [0.1] | Noun [0.1] | Article Noun [0.25] |
      Article Adjs Noun [0.05] | Digit [0.05] | NP PP [0.1] |
      NP RelClause [0.05]

VP -> Verb [0.4] | VP NP [0.35] | VP Adjective [0.05] | VP PP [0.1]
      VP Adverb [0.1]

Adjs -> Adjective [0.8] | Adjective Adjs [0.2]

PP -> Preposition NP [1.0]

RelClause -> RelPro VP [1.0]
```

Algunas frases válidas que produce la gramática: "`mary está triste`", "`eres un robot`" y "`a ella le gusta mary y una buena valla`".

¿Qué pasaría si quisiéramos comprobar si la frase "`mary is sad`" es realmente una oración válida? Podemos usar un **árbol de análisis** para demostrar constructivamente que una cadena de palabras es una frase válida en el idioma dado e incluso calcular la probabilidad de generación de la oración.

![parse_tree](images/parse_tree.png)

La probabilidad de todo el árbol se puede calcular multiplicando las probabilidades de cada transformación de regla individual: `0,9 * 0,1 * 0,05 * 0,05 * 0,4 * 0,05 * 0,3 = 0,00000135`.

Para conservar espacio, también podemos escribir el árbol en forma lineal:

[S [NP [Nombre **maría**]] [VP [VP [Verbo **es**]] [Adjetivo **triste**]]]

Desafortunadamente, la gramática actual **sobregenera**, es decir, crea oraciones que no son gramaticalmente correctas (según el idioma inglés), como "`the valla are john that say`". También **subgenera**, lo que significa que hay oraciones válidas que no genera, como "`él cree que María está triste`".

### Implementación

En el módulo tenemos implementación para gramáticas probabilísticas y no probabilísticas. Ambas implementaciones siguen el mismo formato. Hay funciones para el léxico y las reglas que se pueden combinar para crear un objeto gramatical.

#### No probabilístico

Ejecute la siguiente celda para ver las implementaciones:

In [ ]:
psource(Lexicon, Rules, Grammar)

Construyamos un léxico y una gramática para el lenguaje anterior:

In [2]:
lexicon = Lexicon(
    Verb = "is | say | are",
    Noun = "robot | sheep | fence",
    Adjective = "good | new | sad",
    Adverb = "here | lightly | now",
    Pronoun = "me | you | he",
    RelPro = "that | who | which",
    Name = "john | mary | peter",
    Article = "the | a | an",
    Preposition = "to | in | at",
    Conjunction = "and | or | but",
    Digit = "1 | 2 | 0"
)

print("Lexicon", lexicon)

rules = Rules(
    S = "NP VP | S Conjunction S",
    NP = "Pronoun | Name | Noun | Article Noun \
          | Article Adjs Noun | Digit | NP PP | NP RelClause",
    VP = "Verb | VP NP | VP Adjective | VP PP | VP Adverb",
    Adjs = "Adjective | Adjective Adjs",
    PP = "Preposition NP",
    RelClause = "RelPro VP"
)

print("\nRules:", rules)

Lexicon {'Adverb': ['here', 'lightly', 'now'], 'Verb': ['is', 'say', 'are'], 'Digit': ['1', '2', '0'], 'RelPro': ['that', 'who', 'which'], 'Conjunction': ['and', 'or', 'but'], 'Name': ['john', 'mary', 'peter'], 'Pronoun': ['me', 'you', 'he'], 'Article': ['the', 'a', 'an'], 'Noun': ['robot', 'sheep', 'fence'], 'Adjective': ['good', 'new', 'sad'], 'Preposition': ['to', 'in', 'at']}

Rules: {'RelClause': [['RelPro', 'VP']], 'Adjs': [['Adjective'], ['Adjective', 'Adjs']], 'NP': [['Pronoun'], ['Name'], ['Noun'], ['Article', 'Noun'], ['Article', 'Adjs', 'Noun'], ['Digit'], ['NP', 'PP'], ['NP', 'RelClause']], 'S': [['NP', 'VP'], ['S', 'Conjunction', 'S']], 'VP': [['Verb'], ['VP', 'NP'], ['VP', 'Adjective'], ['VP', 'PP'], ['VP', 'Adverb']], 'PP': [['Preposition', 'NP']]}


Ambas funciones devuelven un diccionario con claves en el lado izquierdo de las reglas. Para el léxico, los valores son los terminales de cada no terminal del lado izquierdo, mientras que para las reglas los valores son los lados derechos como listas.

Ahora podemos usar las variables `léxico` y `reglas` para construir una gramática. Después de haberlo hecho, podemos encontrar las transformaciones de un no terminal (el `Sustantivo`, el `Verbo` y las otras clases básicas **no** cuentan como no terminales adecuados en la implementación). También podemos comprobar si una palabra está en una clase particular.

In [3]:
grammar = Grammar("A Simple Grammar", rules, lexicon)

print("How can we rewrite 'VP'?", grammar.rewrites_for('VP'))
print("Is 'the' an article?", grammar.isa('the', 'Article'))
print("Is 'here' a noun?", grammar.isa('here', 'Noun'))

How can we rewrite 'VP'? [['Verb'], ['VP', 'NP'], ['VP', 'Adjective'], ['VP', 'PP'], ['VP', 'Adverb']]
Is 'the' an article? True
Is 'here' a noun? False


Si la gramática está en la forma normal de Chomsky, podemos llamar a la función de clase `cnf_rules` para obtener todas las reglas en la forma `(X, Y, Z)` para cada regla `X -> Y Z`. Sin embargo, dado que la gramática anterior no está en *CNF*, tenemos que crear una nueva.

In [4]:
E_Chomsky = Grammar("E_Prob_Chomsky", # A Grammar in Chomsky Normal Form
        Rules(
           S = "NP VP",
           NP = "Article Noun | Adjective Noun",
           VP = "Verb NP | Verb Adjective",
        ),
        Lexicon(
           Article = "the | a | an",
           Noun = "robot | sheep | fence",
           Adjective = "good | new | sad",
           Verb = "is | say | are"
        ))

In [5]:
print(E_Chomsky.cnf_rules())

[('S', 'NP', 'VP'), ('VP', 'Verb', 'NP'), ('VP', 'Verb', 'Adjective'), ('NP', 'Article', 'Noun'), ('NP', 'Adjective', 'Noun')]


Finalmente, podemos generar frases aleatorias usando nuestra gramática. La mayoría de ellos serán un completo galimatías y caerán bajo las frases sobregeneradas de la gramática. Esto demuestra que en la gramática las frases válidas son muchas menos que las sobregeneradas.

In [6]:
grammar.generate_random('S')

'sheep that say here mary are the sheep at 2'

#### Probabilístico

Las gramáticas probabilísticas siguen el mismo enfoque. Toman como entrada una cadena, se ensamblan a partir de una gramática y un léxico y pueden generar oraciones aleatorias (dando la probabilidad de la oración). La principal diferencia es que en el léxico tenemos tuplas (terminal, probabilidad) en lugar de cadenas y para las reglas tenemos una lista de tuplas (lista de no terminales, probabilidad) en lugar de una lista de listas de no terminales.

Ejecute las celdas para leer el código:

In [ ]:
psource(ProbLexicon, ProbRules, ProbGrammar)

Construyamos un léxico y reglas para la gramática probabilística:

In [7]:
lexicon = ProbLexicon(
    Verb = "is [0.5] | say [0.3] | are [0.2]",
    Noun = "robot [0.4] | sheep [0.4] | fence [0.2]",
    Adjective = "good [0.5] | new [0.2] | sad [0.3]",
    Adverb = "here [0.6] | lightly [0.1] | now [0.3]",
    Pronoun = "me [0.3] | you [0.4] | he [0.3]",
    RelPro = "that [0.5] | who [0.3] | which [0.2]",
    Name = "john [0.4] | mary [0.4] | peter [0.2]",
    Article = "the [0.5] | a [0.25] | an [0.25]",
    Preposition = "to [0.4] | in [0.3] | at [0.3]",
    Conjunction = "and [0.5] | or [0.2] | but [0.3]",
    Digit = "0 [0.35] | 1 [0.35] | 2 [0.3]"
)

print("Lexicon", lexicon)

rules = ProbRules(
    S = "NP VP [0.6] | S Conjunction S [0.4]",
    NP = "Pronoun [0.2] | Name [0.05] | Noun [0.2] | Article Noun [0.15] \
        | Article Adjs Noun [0.1] | Digit [0.05] | NP PP [0.15] | NP RelClause [0.1]",
    VP = "Verb [0.3] | VP NP [0.2] | VP Adjective [0.25] | VP PP [0.15] | VP Adverb [0.1]",
    Adjs = "Adjective [0.5] | Adjective Adjs [0.5]",
    PP = "Preposition NP [1]",
    RelClause = "RelPro VP [1]"
)

print("\nRules:", rules)

Lexicon {'Noun': [('robot', 0.4), ('sheep', 0.4), ('fence', 0.2)], 'Name': [('john', 0.4), ('mary', 0.4), ('peter', 0.2)], 'Adverb': [('here', 0.6), ('lightly', 0.1), ('now', 0.3)], 'Digit': [('0', 0.35), ('1', 0.35), ('2', 0.3)], 'Adjective': [('good', 0.5), ('new', 0.2), ('sad', 0.3)], 'Pronoun': [('me', 0.3), ('you', 0.4), ('he', 0.3)], 'Article': [('the', 0.5), ('a', 0.25), ('an', 0.25)], 'Preposition': [('to', 0.4), ('in', 0.3), ('at', 0.3)], 'Verb': [('is', 0.5), ('say', 0.3), ('are', 0.2)], 'Conjunction': [('and', 0.5), ('or', 0.2), ('but', 0.3)], 'RelPro': [('that', 0.5), ('who', 0.3), ('which', 0.2)]}

Rules: {'S': [(['NP', 'VP'], 0.6), (['S', 'Conjunction', 'S'], 0.4)], 'RelClause': [(['RelPro', 'VP'], 1.0)], 'VP': [(['Verb'], 0.3), (['VP', 'NP'], 0.2), (['VP', 'Adjective'], 0.25), (['VP', 'PP'], 0.15), (['VP', 'Adverb'], 0.1)], 'Adjs': [(['Adjective'], 0.5), (['Adjective', 'Adjs'], 0.5)], 'PP': [(['Preposition', 'NP'], 1.0)], 'NP': [(['Pronoun'], 0.2), (['Name'], 0.05), (['N

Usemos lo anterior para ensamblar nuestra gramática probabilística y ejecutar algunas consultas simples:

In [8]:
grammar = ProbGrammar("A Simple Probabilistic Grammar", rules, lexicon)

print("How can we rewrite 'VP'?", grammar.rewrites_for('VP'))
print("Is 'the' an article?", grammar.isa('the', 'Article'))
print("Is 'here' a noun?", grammar.isa('here', 'Noun'))

How can we rewrite 'VP'? [(['Verb'], 0.3), (['VP', 'NP'], 0.2), (['VP', 'Adjective'], 0.25), (['VP', 'PP'], 0.15), (['VP', 'Adverb'], 0.1)]
Is 'the' an article? True
Is 'here' a noun? False


Si tenemos una gramática en *CNF*, podemos obtener una lista de todas las reglas. Creemos una gramática en el formulario e imprimamos las reglas *CNF*:

In [9]:
E_Prob_Chomsky = ProbGrammar("E_Prob_Chomsky", # A Probabilistic Grammar in CNF
                             ProbRules(
                                S = "NP VP [1]",
                                NP = "Article Noun [0.6] | Adjective Noun [0.4]",
                                VP = "Verb NP [0.5] | Verb Adjective [0.5]",
                             ),
                             ProbLexicon(
                                Article = "the [0.5] | a [0.25] | an [0.25]",
                                Noun = "robot [0.4] | sheep [0.4] | fence [0.2]",
                                Adjective = "good [0.5] | new [0.2] | sad [0.3]",
                                Verb = "is [0.5] | say [0.3] | are [0.2]"
                             ))

In [10]:
print(E_Prob_Chomsky.cnf_rules())

[('S', 'NP', 'VP', 1.0), ('VP', 'Verb', 'NP', 0.5), ('VP', 'Verb', 'Adjective', 0.5), ('NP', 'Article', 'Noun', 0.6), ('NP', 'Adjective', 'Noun', 0.4)]


Por último, podemos generar oraciones aleatorias a partir de esta gramática. La función `prob_generación` devuelve una tupla (oración, probabilidad).

In [11]:
sentence, prob = grammar.generate_random('S')
print(sentence)
print(prob)

an good sad sheep to 1 is
3.54375e-08


Al igual que con las gramáticas no probabilísticas, ésta en su mayoría genera en exceso. También puedes ver que la probabilidad es muy, muy baja, lo que significa que hay un montón de oraciones generables (en este caso infinitas, ya que tenemos recursividad; observa cómo "VP" puede producir otro "VP", por ejemplo).

## GOLPES

### Descripción general

**La búsqueda de temas inducida por hipervínculos** (o HITS para abreviar) es un algoritmo para la recuperación de información y la clasificación de páginas. Puede leer más sobre la recuperación de información en [text notebook](https://github.com/aimacode/aima-python/blob/master/text.ipynb). Esencialmente, dada una colección de documentos y la consulta de un usuario, dichos sistemas devuelven al usuario los documentos más relevantes para sus necesidades. El algoritmo HITS difiere de muchos otros algoritmos de clasificación similares (como el *Pagerank* de Google) ya que las clasificaciones de las páginas en este algoritmo dependen de la consulta dada. Esto significa que para cada nueva consulta las páginas de resultados deben calcularse nuevamente. Este coste puede resultar prohibitivo para muchos motores de búsqueda modernos, por lo que muchos evitan este enfoque.

HITS primero encuentra una lista de páginas relevantes para la consulta y luego agrega páginas que enlazan o están enlazadas desde estas páginas. Una vez construido el conjunto, definimos dos valores para cada página. **Autoridad** en la consulta, el grado de páginas del conjunto relevante que enlazan con ella y **centro** de la consulta, el grado en que apunta a páginas autorizadas en el conjunto. Como no queremos simplemente contar el número de enlaces de una página a otras páginas, sino que también queremos tener en cuenta la calidad de las páginas enlazadas, actualizamos los valores de centro y autoridad de una página de la siguiente manera, hasta convergencia:

* Puntuación del centro = La suma de las puntuaciones de autoridad de las páginas a las que enlaza.

* Puntuación de autoridad = La suma de las puntuaciones centrales de las páginas desde las que está vinculado.

Por lo tanto, cuanto mayor sea la calidad de las páginas a las que está vinculada una página, mayores serán sus puntuaciones.

Luego normalizamos las puntuaciones dividiendo cada puntuación por la suma de los cuadrados de las puntuaciones respectivas de todas las páginas. Cuando los valores convergen, devolvemos las páginas mejor valoradas. Tenga en cuenta que debido a que normalizamos los valores, se garantiza que el algoritmo convergerá.

### Implementación

El código fuente del algoritmo se proporciona a continuación:

In [ ]:
psource(HITS)

Primero compilamos la colección de páginas como se mencionó anteriormente. Luego, inicializamos las puntuaciones de autoridad y centro para cada página y finalmente actualizamos y normalizamos los valores hasta la convergencia.

Una descripción general rápida de las funciones auxiliares que utilizamos:

* `relevant_pages`: Devuelve páginas relevantes de `pagesIndex` dada una consulta.

* `expand_pages`: Agrega a la colección páginas vinculadas hacia y desde las `páginas` dadas.

* `normalizar`: normaliza las puntuaciones de autoridad y centro.

* `ConvergenceDetector`: una clase que verifica la convergencia, manteniendo un historial de las puntuaciones de las páginas y verificando si cambian o no.

* `Página`: La plantilla para páginas. Almacena la dirección, puntuaciones de autoridad/centro y enlaces de entrada/salida.

### Ejemplo

Antes de comenzar, necesitamos definir una lista de páginas de muestra en las que trabajar. Las páginas son `pA`, `pB`, etc. y su texto viene dado por `testHTML` y `testHTML2`. La clase `Page` toma como argumentos los enlaces de entrada y salida como listas. Para la página "A", los enlaces de entrada son "B", "C" y "E", mientras que el único enlace de salida es "D".

También necesitamos configurar las variables globales `nlp` `pageDict`, `pagesIndex` y `pagesContent`.

In [12]:
testHTML = """Like most other male mammals, a man inherits an
            X from his mom and a Y from his dad."""
testHTML2 = "a mom and a dad"

pA = Page('A', ['B', 'C', 'E'], ['D'])
pB = Page('B', ['E'], ['A', 'C', 'D'])
pC = Page('C', ['B', 'E'], ['A', 'D'])
pD = Page('D', ['A', 'B', 'C', 'E'], [])
pE = Page('E', [], ['A', 'B', 'C', 'D', 'F'])
pF = Page('F', ['E'], [])

nlp.pageDict = {pA.address: pA, pB.address: pB, pC.address: pC,
                pD.address: pD, pE.address: pE, pF.address: pF}

nlp.pagesIndex = nlp.pageDict

nlp.pagesContent ={pA.address: testHTML, pB.address: testHTML2,
                   pC.address: testHTML, pD.address: testHTML2,
                   pE.address: testHTML, pF.address: testHTML2}

Ahora podemos ejecutar el algoritmo HITS. Nuestra consulta será 'mamíferos' (tenga en cuenta que si bien el contenido del HTML no importa, debe incluir las palabras de consulta o de lo contrario no se seleccionará ninguna página en el primer paso).

In [13]:
HITS('mammals')
page_list = ['A', 'B', 'C', 'D', 'E', 'F']
auth_list = [pA.authority, pB.authority, pC.authority, pD.authority, pE.authority, pF.authority]
hub_list = [pA.hub, pB.hub, pC.hub, pD.hub, pE.hub, pF.hub]

Veamos cómo se puntuaron las páginas:

In [14]:
for i in range(6):
    p = page_list[i]
    a = auth_list[i]
    h = hub_list[i]
    
    print("{}: total={}, auth={}, hub={}".format(p, a + h, a, h))

A: total=0.7696163397038682, auth=0.5583254178509696, hub=0.2112909218528986
B: total=0.7795962360479536, auth=0.23657856688600404, hub=0.5430176691619495
C: total=0.8204496913590655, auth=0.4211098490570872, hub=0.3993398423019784
D: total=0.6316647735856309, auth=0.6316647735856309, hub=0.0
E: total=0.7078245882072104, auth=0.0, hub=0.7078245882072104
F: total=0.23657856688600404, auth=0.23657856688600404, hub=0.0


La puntuación máxima es 0,82 por "C". Esta es la página más relevante según el algoritmo. Puede ver que las páginas a las que enlaza, "A" y "D", tienen las dos puntuaciones de autoridad más altas (por lo tanto, "C" tiene una puntuación central alta) y las páginas desde las que enlaza, "B" y "E". , tienen las puntuaciones centrales más altas (por lo que "C" tiene una puntuación de autoridad alta). Combinando estos dos hechos, obtenemos que "C" es la página más relevante. Vale la pena señalar que no importa si la página dada contiene las palabras de consulta, solo que enlace y esté enlazada desde páginas de alta calidad.

## RESPUESTA A PREGUNTAS

**Respuesta de preguntas** es un tipo de sistema de recuperación de información, donde tenemos una pregunta en lugar de una consulta y en lugar de documentos relevantes queremos que la computadora devuelva una oración, frase o palabra corta que responda a nuestra pregunta. Para comprender mejor el concepto de sistemas de respuesta a preguntas, primero puede leer la sección "Modelos de texto" y "Recuperación de información" de [text notebook](https://github.com/aimacode/aima-python/blob/master/text.ipynb).

Un ejemplo típico de tal sistema es "AskMSR" (Banko *et al.*, 2002), un sistema para responder preguntas que funcionó admirablemente frente a algoritmos más sofisticados. La idea básica detrás de esto es que muchas preguntas ya han sido respondidas en la web en numerosas ocasiones. El sistema no sabe mucho sobre verbos, conceptos o incluso qué es un sustantivo. Conoce alrededor de 15 tipos diferentes de preguntas y cómo se pueden escribir como consultas. Puede reescribir [¿Quién era el segundo al mando de George Washington?] como la consulta [\* era el segundo al mando de George Washington] o [el segundo al mando de George Washington era \*].

Después de reescribir las preguntas, emite estas consultas y recupera el texto breve sobre los términos de la consulta. Luego divide el resultado en 1, 2 o 3 gramos. También se aplican filtros para aumentar las posibilidades de una respuesta correcta. Si la consulta comienza con "quién", filtramos por nombres, si comienza con "cuántos", filtramos por números, etc. También podemos filtrar las palabras que aparecen en la consulta. Para la consulta anterior, la respuesta "George Washington" es incorrecta, aunque es muy posible que los 2 gramos aparezcan mucho en los términos de la consulta.

Finalmente, los diferentes resultados se ponderan por la generalidad de las consultas. El resultado de la consulta booleana general [George Washington O segundo al mando] pesa menos que la consulta más específica [el segundo al mando de George Washington era \*]. Como respuesta, devolvemos el ngrama mejor clasificado.

## CICLO DE ANÁLISIS

### Descripción general

El análisis sintáctico (o **análisis**) de una oración es el proceso de descubrir la estructura sintáctica de la oración de acuerdo con las reglas de una gramática. Hay dos enfoques principales para el análisis. *De arriba hacia abajo*, comenzamos con el símbolo inicial y construimos un árbol de análisis con las palabras dadas como hojas, y *de abajo hacia arriba*, donde comenzamos a partir de las palabras dadas y construimos un árbol que tiene el símbolo inicial como raíz . Ambos enfoques implican "adivinar" con anticipación, por lo que es muy posible que lleve mucho tiempo analizar una oración (una suposición incorrecta significa mucho retroceso). Afortunadamente, se dedica mucho esfuerzo a analizar subcadenas ya analizadas, por lo que podemos seguir un enfoque de programación dinámica para almacenar y reutilizar estos análisis en lugar de recalcularlos. El *Algoritmo de análisis CYK* (llamado así por sus inventores, Cocke, Younger y Kasami) utiliza esta técnica para analizar oraciones de una gramática en *Forma normal de Chomsky*.

El algoritmo CYK devuelve una matriz *M x N x N* (llamada *P*), donde *N* es el número de palabras de la oración y *M* el número de símbolos no terminales de la gramática. Cada elemento de esta matriz muestra la probabilidad de que una subcadena se transforme a partir de un no terminal en particular. Para encontrar el análisis más probable de la oración, se requiere una búsqueda en la matriz resultante. Los algoritmos heurísticos de búsqueda funcionan bien en este espacio y podemos derivar las heurísticas de las propiedades de la gramática.

En resumen, el algoritmo funciona así: hay un bucle externo que determina la longitud de la subcadena. Luego, el algoritmo recorre las palabras de la oración. Para cada palabra, recorre nuevamente todas las palabras a su derecha hasta la longitud del primer bucle. La subcadena con la que trabajará en esta iteración son las palabras de la palabra del segundo ciclo con la longitud del primer ciclo. Finalmente, recorre todas las reglas de la gramática y actualiza la probabilidad de la subcadena para cada no terminal del lado derecho.

### Implementación

La implementación toma como entrada una lista de palabras y una gramática probabilística (de la clase `ProbGrammar` detallada anteriormente) en CNF y devuelve la tabla/diccionario *P*. La clave de un elemento en *P* es una tupla en la forma `(No terminal, inicio de subcadena, longitud de subcadena)` y el valor es una probabilidad. Por ejemplo, para la oración "el mono está bailando" y la subcadena "el mono" un elemento puede ser `('NP', 0, 2): 0.5`, que significa las dos primeras palabras (la subcadena del índice 0 y longitud 2) tienen una probabilidad de 0,5 de provenir del terminal `NP`.

Antes de continuar, puedes echar un vistazo al código fuente ejecutando la siguiente celda:

In [ ]:
psource(CYK_parse)

Al actualizar la probabilidad de una subcadena, elegimos el máximo de la actual y la probabilidad de que la subcadena se divida en dos partes: una de la palabra del segundo bucle con longitud del tercer bucle y la otra desde el final de la primera parte hasta el final. resto de la longitud del primer bucle.

### Ejemplo

Construyamos una gramática probabilística en CNF:

In [15]:
E_Prob_Chomsky = ProbGrammar("E_Prob_Chomsky", # A Probabilistic Grammar in CNF
                             ProbRules(
                                S = "NP VP [1]",
                                NP = "Article Noun [0.6] | Adjective Noun [0.4]",
                                VP = "Verb NP [0.5] | Verb Adjective [0.5]",
                             ),
                             ProbLexicon(
                                Article = "the [0.5] | a [0.25] | an [0.25]",
                                Noun = "robot [0.4] | sheep [0.4] | fence [0.2]",
                                Adjective = "good [0.5] | new [0.2] | sad [0.3]",
                                Verb = "is [0.5] | say [0.3] | are [0.2]"
                             ))

Ahora veamos la tabla de probabilidades de la frase "el robot es bueno":

In [16]:
words = ['the', 'robot', 'is', 'good']
grammar = E_Prob_Chomsky

P = CYK_parse(words, grammar)
print(P)

defaultdict(<class 'float'>, {('Adjective', 1, 1): 0.0, ('NP', 0, 3): 0.0, ('Verb', 1, 1): 0.0, ('NP', 0, 2): 0.12, ('S', 1, 2): 0.0, ('Article', 2, 1): 0.0, ('NP', 3, 1): 0.0, ('S', 1, 3): 0.0, ('Adjective', 1, 3): 0.0, ('VP', 0, 4): 0.0, ('Article', 0, 3): 0.0, ('Adjective', 1, 2): 0.0, ('Verb', 1, 2): 0.0, ('Adjective', 0, 2): 0.0, ('Article', 0, 1): 0.5, ('VP', 1, 1): 0.0, ('Verb', 0, 2): 0.0, ('Adjective', 0, 3): 0.0, ('VP', 1, 2): 0.0, ('Verb', 0, 3): 0.0, ('NP', 2, 2): 0.0, ('S', 2, 2): 0.0, ('NP', 1, 3): 0.0, ('VP', 1, 3): 0.0, ('Adjective', 3, 1): 0.5, ('Adjective', 0, 1): 0.0, ('NP', 1, 2): 0.0, ('Verb', 0, 1): 0.0, ('S', 0, 3): 0.0, ('NP', 1, 1): 0.0, ('NP', 2, 1): 0.0, ('S', 0, 2): 0.0, ('Noun', 1, 2): 0.0, ('S', 0, 4): 0.015, ('Noun', 1, 3): 0.0, ('Noun', 3, 1): 0.0, ('Noun', 2, 2): 0.0, ('NP', 0, 4): 0.0, ('VP', 2, 2): 0.125, ('Noun', 2, 1): 0.0, ('Noun', 1, 1): 0.4, ('VP', 0, 3): 0.0, ('Article', 1, 2): 0.0, ('Article', 1, 1): 0.0, ('VP', 2, 1): 0.0, ('Adjective', 2, 1):

Se devuelve un objeto `defaultdict` (`defaultdict` es básicamente un diccionario pero con un valor/tipo predeterminado). Las claves son tuplas en la forma mencionada anteriormente y los valores son las probabilidades correspondientes. La mayoría de los elementos/análisis tienen una probabilidad de 0. Filtrémoslos para ver mejor los análisis que importan.

In [17]:
parses = {k: p for k, p in P.items() if p >0}

print(parses)

{('Noun', 1, 1): 0.4, ('VP', 2, 2): 0.125, ('Adjective', 3, 1): 0.5, ('S', 0, 4): 0.015, ('Article', 0, 1): 0.5, ('NP', 0, 2): 0.12, ('Verb', 2, 1): 0.5}


El elemento `('Artículo', 0, 1): 0,5` significa que el primer elemento proviene del no terminal `Artículo` con una probabilidad de 0,5. Un elemento más complicado, uno con dos palabras, es `('NP', 0, 2): 0.12` que cubre las dos primeras palabras. La probabilidad de que la subcadena "el robot" provenga del no terminal "NP" es 0,12. Intentemos seguir las transformaciones de `NP` a las palabras dadas (de arriba hacia abajo) para asegurarnos de que este sea realmente el caso:

1. La probabilidad de que "NP" se transforme en "Sustantivo de artículo" es 0,6.

2. La probabilidad de que "Artículo" se transforme en "el" es 0,5 (probabilidad total = 0,6*0,5 = 0,3).

3. La probabilidad de que "Sustantivo" se transforme en "robot" es 0,4 (total = 0,3*0,4 = 0,12).

Por tanto, la probabilidad total de la transformación es 0,12.

Observe cómo la probabilidad para toda la cadena (dada por la clave `('S', 0, 4)`) es 0,015. Esto significa que el análisis más probable de la oración tiene una probabilidad de 0,015.

## ANÁLISIS DE GRÁFICOS

### Descripción general

Veamos ahora un algoritmo de análisis de gráficos más general. Dadas una gramática no probabilística y una oración, este algoritmo construye un árbol de análisis de arriba hacia abajo, con las palabras de la oración como hojas. Funciona con un enfoque de programación dinámica, creando un gráfico para almacenar análisis de subcadenas para que no tenga que analizarlas nuevamente (al igual que el algoritmo CYK). Cada no terminal, comenzando desde S, es reemplazado por sus reglas del lado derecho en el gráfico, hasta que terminemos con los análisis correctos.

### Implementación

Un análisis tiene la forma `[inicio, fin, no terminal, subárbol, transformación esperada]`, donde `subárbol` es un árbol con el correspondiente `no terminal` como raíz y `esperado- La transformación "es una regla del lado derecho del" no terminal ".

El análisis del gráfico se implementa en una clase, "Gráfico". Se inicializa con una gramática y puede devolver la lista de todos los análisis de una oración con la función "análisis".

El gráfico es una lista de listas. Las listas corresponden a las longitudes de las subcadenas (incluida la cadena vacía), de principio a fin. Cuando decimos "un punto en el gráfico", nos referimos a una lista de cierta longitud.

Un resumen rápido de las funciones de clase:

* `parses`: Devuelve una lista de análisis para una oración determinada. Si la oración no se puede analizar, devolverá una lista vacía. Inicializa el proceso llamando a "parse" desde el símbolo inicial.


* `parse`: analiza la lista de palabras y crea el gráfico.


* `add_edge`: Agrega otro borde al gráfico en un punto determinado. Además, examina si el borde se extiende o predice otro borde. Si el borde en sí no espera una transformación, extenderá otros bordes y, de lo contrario, predecirá los bordes.


* `escáner`: Dada una palabra y un punto en el gráfico, extiende los bordes que esperaban una transformación que puede resultar en la palabra dada. Por ejemplo, si la palabra 'el' es un 'Artículo' y estamos examinando dos bordes en el punto de un gráfico, uno esperando un 'Artículo' y el otro un 'Verbo', el primero se extenderá mientras que el segundo no lo hará.


* `predictor`: si un borde no puede extender otros bordes (porque está esperando una transformación en sí), agregaremos al gráfico reglas/transformaciones que pueden ayudar a extender el borde. Las nuevas aristas provienen del lado derecho de las reglas de transformación esperadas. Por ejemplo, si un borde espera la transformación 'Adjetivo Sustantivo', agregaremos al gráfico un borde para cada regla del lado derecho del 'Adjetivo' no terminal.


* `extender`: Extiende los bordes dado un borde (llamado `E`). Si el no terminal de `E` es la misma que la transformación esperada de otra arista (llamémosla `A`), agregue al gráfico una nueva arista con el no terminal de `A` y las transformaciones de `A `menos el no terminal que coincidió con el no terminal de `E`. Por ejemplo, si una arista "E" tiene "Artículo" como no terminal y no espera ninguna transformación, necesitamos ver qué aristas puede extender. Examinemos el borde `N`. Esto espera una transformación de 'Noun Verb'. 'Sustantivo' no coincide con 'Artículo', así que seguimos adelante. Otra arista, "A", espera una transformación de "Article Noun" y tiene un no terminal de "NP". ¡Tenemos un partido! Se agregará una nueva arista con 'NP' como su no terminal (el no terminal de 'A') y 'Sustantivo' como la transformación esperada (el resto de la transformación esperada de 'A').

Puede ver el código fuente ejecutando la siguiente celda:

In [ ]:
psource(Chart)

### Ejemplo

Usaremos la gramática `E0` para analizar la oración "el hedor está en 2 2".

Primero necesitamos construir un objeto "Gráfico":

In [18]:
chart = Chart(nlp.E0)

Y luego simplemente llamamos a la función `parses`:

In [19]:
print(chart.parses('the stench is in 2 2'))

[[0, 6, 'S', [[0, 2, 'NP', [('Article', 'the'), ('Noun', 'stench')], []], [2, 6, 'VP', [[2, 3, 'VP', [('Verb', 'is')], []], [3, 6, 'PP', [('Preposition', 'in'), [4, 6, 'NP', [('Digit', '2'), ('Digit', '2')], []]], []]], []]], []]]


Puede ver qué bordes se agregan estableciendo el argumento de inicialización opcional "trace" en verdadero.

In [ ]:
chart_trace = Chart(nlp.E0, trace=True)
chart_trace.parses('the stench is in 2 2')

Intentemos analizar una oración que la gramática no reconoce:

In [20]:
print(chart.parses('the stench 2 2'))

[]


Se devolvió una lista vacía.